In [ ]:
!pip install google-genai

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
import concurrent.futures
from google import genai

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()

# Set the model for our agents
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define Subagents (to be run in parallel) ---

def agent_sentiment(text: str) -> str:
    """Subagent A: Analyzes the emotional tone."""
    print("-> [Sentiment Agent] Analyzing...")
    prompt = f"Analyze the sentiment of this customer feedback. Is it positive, negative, or neutral? Keep it to one sentence.\n\nFeedback: {text}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

def agent_keywords(text: str) -> str:
    """Subagent B: Extracts main topics."""
    print("-> [Keyword Agent] Extracting...")
    prompt = f"Extract 3 to 5 key topics or keywords from this feedback as a comma-separated list.\n\nFeedback: {text}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

def agent_urgency(text: str) -> str:
    """Subagent C: Determines priority level."""
    print("-> [Urgency Agent] Assessing...")
    prompt = f"Assess the urgency of this feedback. Reply strictly with High, Medium, or Low, followed by a one-sentence reason.\n\nFeedback: {text}"
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

# --- 2. Define Synthesizer Agent ---

def agent_synthesizer(sentiment: str, keywords: str, urgency: str) -> str:
    """Gathers the intermediate outputs into a final consolidated response."""
    print("-> [Synthesizer Agent] Compiling final report...")
    prompt = f"""Synthesize the following subagent reports into one concise, professional customer feedback summary.
    Use clear Markdown headers.

    Sentiment Report: {sentiment}
    Keywords Report: {keywords}
    Urgency Report: {urgency}
    """
    return client.models.generate_content(model=MODEL_ID, contents=prompt).text

# --- 3. Define the Orchestrator ---

def parallel_workflow_agent(user_prompt: str) -> str:
    """
    The orchestrator agent.
    It fans out the prompt to subagents concurrently and gathers the results.
    """
    print("Starting Parallel Workflow...\n")

    # Use ThreadPoolExecutor to run tasks concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks to the thread pool (they start executing immediately)
        future_sentiment = executor.submit(agent_sentiment, user_prompt)
        future_keywords  = executor.submit(agent_keywords, user_prompt)
        future_urgency   = executor.submit(agent_urgency, user_prompt)

        # .result() waits for the thread to finish and gets the output.
        # Since they run concurrently, the total wait time is just the time of the slowest agent.
        sentiment_result = future_sentiment.result()
        keywords_result  = future_keywords.result()
        urgency_result   = future_urgency.result()

    print("\n--- All parallel tasks complete. Synthesizing... ---\n")

    # Synthesize the intermediate outputs
    final_result = agent_synthesizer(sentiment_result, keywords_result, urgency_result)

    return final_result

# --- Execution ---
if __name__ == "__main__":
    # Example customer feedback
    feedback = (
        "I've been trying to log into my account for three days, but the password "
        "reset email never arrives! I need to access my billing statements immediately "
        "for my taxes. This is incredibly frustrating and your support line hung up on me."
    )

    # Trigger the parallel agent
    final_output = parallel_workflow_agent(feedback)

    print("\n=== Final Consolidated Response ===")
    print(final_output)

Starting Parallel Workflow...

-> [Sentiment Agent] Analyzing...
-> [Keyword Agent] Extracting...
-> [Urgency Agent] Assessing...

--- All parallel tasks complete. Synthesizing... ---

-> [Synthesizer Agent] Compiling final report...

=== Final Consolidated Response ===
## Customer Feedback Summary

### Overall Sentiment & Key Issues
This feedback indicates extreme customer frustration and dissatisfaction, stemming primarily from critical account access issues. The user is experiencing persistent login difficulties and repeated failures with password reset attempts. This is compounded by concerns regarding billing statements and a perceived failure in previous customer support interactions.

### Urgency & Impact
The urgency level is **High**. The user is completely locked out of their account and requires immediate access for critical tax-related purposes. This situation is further exacerbated by an reported inability to resolve the issue through prior support channels.
